In [ ]:
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
import itertools
import pandas as pd
from scipy.signal import filtfilt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation

### Data functions

In [ ]:
def data_average(df):
    return df.mean()

def quaternion_average(df):
    # function taken from
    # https://stackoverflow.com/questions/12374087/average-of-multiple-quaternions
    A = df.transpose().dot(df)
    w, v = np.linalg.eig(A)
    q = v[:, w.argmax()].real
    q = -q if q[0] < 0 else q
    return q

def downsample(data, time_vector, average_function, selected_columns):
    def insert_row(data, row, labels=None):
        return data.append(pd.Series(row, labels), ignore_index=True)
    
    current_time_index = 0
    downsampled_data = pd.DataFrame(columns=selected_columns)
    
    for i in range(len(time_vector)):
        t = time_vector.iloc[i]
        start_time = current_time_index
        
        while current_time_index < data.shape[0] and data['relative_time'].iloc[current_time_index] < t:
            current_time_index = current_time_index + 1
        stop_time = current_time_index
        
        if stop_time != start_time:
            average_data = average_function(data.iloc[start_time:stop_time][selected_columns])
            downsampled_data = insert_row(downsampled_data, average_data, selected_columns)
        
    return downsampled_data

def remove_offset(data, samples):
    def get_offset(column, samples):
        return np.sum(column.head(samples)) / float(samples)
    corrected_data = data.apply(lambda x: x - get_offset(x, samples), axis=0)
    return corrected_data

def filter_data(data, order, a=1):
    b = [1.0 / order] * order
    return data.apply(lambda x: filtfilt(b, a, x), axis=0)

### Transform functions

In [ ]:
def get_knife_transform(fruit_quality):
    if fruit_quality in knife_extension:
        extension = knife_extension[fruit_quality]
    else:
        extension = knife_extension["default"]
    T = np.eye(4)
    T[2,3] = extension
    return T

def get_hom_transform(df_row, position_headers, orientation_headers):
    T = np.eye(4)
    T[:3,3] = df_row[position_headers]
    T[:3,:3] = Rotation.from_quat(df_row[orientation_headers]).as_matrix()
    return T

def get_hom_transform_inv(T):
    T_inv = np.eye(4)
    T_inv[:3,:3] = np.transpose(T[:3,:3])
    T_inv[:3,3] = -T_inv[:3,:3].dot(T[:3,3])
    return T_inv

def transform_data(df_row, knife_tip_T, ft_headers, position_headers, orientation_headers):
    world_T_knife = get_hom_transform(df_row, position_headers, orientation_headers)
    FT_T_knife = np.matmul(FT_T_world, np.matmul(world_T_knife, knife_tip_T))
    df_row[position_headers] = FT_T_knife[:3,3]
    df_row[orientation_headers] = Rotation.from_matrix(FT_T_knife[:3,:3]).as_quat()
    knife_T_FT = get_hom_transform_inv(FT_T_knife)
    
    # correct for steepness too
    desired_x_direction = np.cross([0,0,1], FT_T_knife[:3,2])
    desired_x_direction = desired_x_direction / np.linalg.norm(desired_x_direction)
    angle = np.arctan2(np.linalg.norm(np.cross(FT_T_knife[:3,0],desired_x_direction)), 
                       np.dot(FT_T_knife[:3,0],desired_x_direction))
    if FT_T_knife[2,0] > 0:
        angle = -angle
    knife_T_cutting_direction = Rotation.from_euler("z", angle, degrees=False).as_matrix()
    
    df_row[ft_headers] = np.matmul(knife_T_cutting_direction.transpose(),knife_T_FT[:3,:3]).dot(df_row[ft_headers])
    return df_row

### Plot functions

In [ ]:
def plot_data(x, y, header=""):
    fig = make_subplots(rows=y.shape[1], cols=1,x_title='Time',)
    
    for index in range(y.shape[1]):
        fig.append_trace(go.Scatter(
            x=x,
            y=y.iloc[:,index],
            name=y.columns[index],
        ), row=index+1, col=1)

    fig.update_layout(height=600, width=600, title_text=header)
    fig.show()

### Other

In [ ]:
def export_data(runs, timeseries, folder_name):
    export_folder = join('..', 'data', folder_name)
    
    if not os.path.isdir(export_folder):
        os.makedirs(export_folder)
    
    for i, r in enumerate(runs):
        filename = os.path.split(r)
        name = '_'.join(filename[0].split('/')[-2:] + [filename[1]])
        timeseries[i].to_csv(join(export_folder, name + '.csv'))
    
    

# Data choice

In [ ]:
experiment = 'december'

In [ ]:
fruits = ['orange']

In [ ]:
cut_qualities = ['good'] # good / shallow / deep

In [ ]:
frames = ['ExactoKnife']

In [ ]:
force_components = ['force'] # force and/or torque

In [ ]:
knife_extension = {"default": 0.003, "banana-deep": 0.006}
world_T_FT = np.array([[-0.0032, 1.0000, 0.0062, 0.0527],
                   [-1.0000, -0.0032, -0.0090, 0.9641],
                   [-0.0090, -0.0062, 0.9999, 0.3465],
                   [0, 0, 0, 1.0000]])
FT_T_world = get_hom_transform_inv(world_T_FT)

# Processing options

In [ ]:
remove_sensor_offset = True
if remove_sensor_offset:
    samples = 40


In [ ]:
filter_force = True
if filter_force:
    force_filter_order = 20

In [ ]:
transform_all = True # this will transform the force into the knife frame and the knife positions into the FT frame

# Data extraction

In [ ]:
reader = Reader()

In [ ]:
data_folder = join('..', 'data', 'raw_data', experiment)
print(data_folder)

In [ ]:
folders  = {cq: [join(data_folder, f, cq) for f in fruits] for cq in cut_qualities}
print(folders)

In [ ]:
all_runs = {cq: list(itertools.chain.from_iterable([x[0] for x in os.walk(f, followlinks=True)][1:] for f in folder)) for cq, folder in folders.items()}
print(all_runs)

In [ ]:
opt_position_header = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames))
opt_orient_header = list(itertools.chain.from_iterable((f + '_qx', f + '_qy', f + '_qz', f + '_qw') for f in frames))

In [ ]:
ft_desired_header = list(itertools.chain.from_iterable((v + '_x', v + '_y', v + '_z') for v in force_components))

# Transformation

In [ ]:
timeseries = []

for cq, runs in all_runs.items():
    for r in runs:
        print("Processing run " + r)
        reader.play(r)
        timestamp = reader.data['ft_sensor']['relative_time']
        timestamp.reset_index(drop= True,inplace=True)

        # exctract force data
        force_data = reader.data['ft_sensor'][ft_desired_header].reset_index(drop=True)
        if remove_sensor_offset:
            force_data = remove_offset(force_data, samples)
        if filter_force:
            force_data = filter_data(force_data, force_filter_order)
        #plot_data(timestamp,force_data)

        # downsample optitrack data
        opt_position_data = downsample(reader.data['optitrack'], timestamp, data_average, opt_position_header)
        opt_orient_data = downsample(reader.data['optitrack'], timestamp, quaternion_average, opt_orient_header)
        #plot_data(timestamp, opt_position_data[opt_position_header])
        
        # merge the data and store
        merge_data = pd.concat([timestamp, opt_position_data, opt_orient_data, force_data], axis=1)
        merge_data = merge_data.dropna()
        
        # transform optitrack to FT frame and force to knife frame
        if transform_all:
            merge_data.apply(lambda x: transform_data_2(x, get_knife_transform("-".join([fruits[0], cq])), 
                                                      ft_desired_header, opt_position_header, opt_orient_header), axis=1)
        #plot_data(timestamp, merge_data[opt_position_header])
        #plot_data(timestamp, merge_data[ft_desired_header])
        
        timeseries.append(merge_data)
        
    export_data(runs, timeseries, "processed_transformed_data")

# Segmentation

In [ ]:
fig = make_subplots(rows=3, cols=1,x_title='time [s]')
fig1 = make_subplots(rows=3, cols=1,x_title='time [s]')
fig2 = make_subplots(rows=3, cols=1,x_title='relative displacement from cut start [m]')

timeseries2 = [] 
merge_data2 = pd.DataFrame()

for i, r in enumerate(runs):
    gradient = [(b - a) for a, b in
                        zip(timeseries[i]["force_x"][:-1], timeseries[i]["force_x"][1:])]
    max_index = gradient.index(max(gradient))
    min_index = gradient.index(min(gradient))
    ignore_edges_length = 0
    data_segmented = timeseries[i].loc[max_index + ignore_edges_length:min_index - ignore_edges_length]
    data_segmented.reset_index(inplace=True)
    
    total_time = data_segmented["relative_time"].iloc[-1] - data_segmented.loc[0, "relative_time"]
    data_segmented.loc[:, "relative_time"] = (data_segmented["relative_time"] -
                                                      data_segmented.loc[
                                                          0, "relative_time"]) / total_time * 5
    
    for column in opt_position_header:
        data_segmented.loc[:, column] -= data_segmented.loc[0, column]
    
    #plot_data(data_segmented["relative_time"], data_segmented[ft_desired_header])
    #plot_data(data_segmented["relative_time"], data_segmented[opt_position_header])
    
    angle = np.arctan(data_segmented["ExactoKnife_x"].iloc[-20] / data_segmented["ExactoKnife_y"].iloc[-20])
    R = Rotation.from_euler("z", angle).as_matrix()
    data_segmented.loc[:, "ExactoKnife_x"] = [(R.dot(x))[0] for x in
                                              zip(data_segmented["ExactoKnife_x"],
                                                  data_segmented["ExactoKnife_y"],
                                                  data_segmented["ExactoKnife_z"])]
    data_segmented.loc[:, "ExactoKnife_y"] = [(R.dot(x))[1] for x in
                                              zip(data_segmented["ExactoKnife_x"],
                                                  data_segmented["ExactoKnife_y"],
                                                  data_segmented["ExactoKnife_z"])]
    
    for i, pos in enumerate(opt_position_header):
        fig.append_trace(go.Scatter(
                x=data_segmented["relative_time"],
                y=data_segmented[pos],
                ), row=i+1, col=1)
    for i, force in enumerate(ft_desired_header):
        fig1.append_trace(go.Scatter(
                x=data_segmented["relative_time"],
                y=data_segmented[force],
                ), row=i+1, col=1)
        
    displacement = [np.linalg.norm(x) for x in
                            zip(data_segmented["ExactoKnife_x"], data_segmented["ExactoKnife_y"],
                                data_segmented["ExactoKnife_z"])]
    
    data_segmented["displacement"] = displacement
    timeseries2.append(data_segmented)
    
    for i, force in enumerate(ft_desired_header):
        fig2.append_trace(go.Scatter(
                x=displacement,
                y=data_segmented[force],
                mode='markers',
                showlegend=False,
                ), row=i+1, col=1)
        
    #merge_data2 = pd.concat([merge_data2, data_segmented], axis=0)

#print(merge_data2)
        
export_data(runs, timeseries2, "segmented_data")
#export_data(runs, [merge_data2], "concat_data")

model_x = pd.read_csv(join(data_folder,"model_x.csv"), names=["dis","y","y+","y-"])
model_y = pd.read_csv(join(data_folder,"model_y.csv"), names=["dis","y","y+","y-"])
model_z = pd.read_csv(join(data_folder,"model_z.csv"), names=["dis","y","y+","y-"])



fig.update_yaxes(title_text="position_x [m]", row=1, col=1)
fig.update_yaxes(title_text="position_y [m]", row=2, col=1)
fig.update_yaxes(title_text="position_z [m]", row=3, col=1)
fig.update_layout(height=600, width=800, title_text='-'.join([fruits[0],cut_qualities[0]]),showlegend=False)
fig.show()


fig1.update_yaxes(title_text="force_x [N]", row=1, col=1)
fig1.update_yaxes(title_text="force_y [N]", row=2, col=1)
fig1.update_yaxes(title_text="force_z [N]", row=3, col=1)
fig1.update_layout(height=600, width=800, title_text='-'.join([fruits[0],cut_qualities[0]]),showlegend=False)
fig1.show()

for i, model in enumerate([model_x]):
    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y"],
                    line=dict(color='rgb(0,0,0)', width=3),
                    name="y_predicted",
                    ), row=i+1, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y+"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    name="y_predicted + variance",
                    ), row=i+1, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y-"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    name="y_predicted - variance",
                    ), row=i+1, col=1)

for i, model in enumerate([model_y, model_z]):
    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y"],
                    line=dict(color='rgb(0,0,0)', width=3),
                    showlegend=False,
                    ), row=i+2, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y+"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    showlegend=False,
                    ), row=i+2, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y-"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    showlegend=False,
                    ), row=i+2, col=1)

fig2.update_yaxes(title_text="force_x [N]", row=1, col=1)
fig2.update_yaxes(title_text="force_y [N]", row=2, col=1)
fig2.update_yaxes(title_text="force_z [N]", row=3, col=1)
fig2.update_layout(height=800, width=1000, title_text='-'.join([fruits[0],cut_qualities[0]]))
fig2.show()
    
    

# Export preprocessed data

In [ ]:
if not os.path.isdir(export_folder):
    os.makedirs(export_folder)

In [ ]:
for i, r in enumerate(runs):
    filename = os.path.split(r)
    name = '_'.join(filename[0].split('/')[-2:] + [filename[1]])
    timeseries[i].to_csv(join(export_folder, name + '.csv'))